In [20]:
from brian2 import *
import numpy as np

# Introduction

Dendritic spines are an essential structure in dlPFC pyramidal neurons for regulating neuromodulators. Decreased populations of such spines have, for instance, been associated with schizophrenia and other neurological disorders. Studying their effect on excitatory neurotransmission is the goal of this project. Specifically, we want to look into the results of Cornejo, et al. 2022 which finds that although spines follow the voltages of their parent dendrites during action potentials, spines have different voltages during sub-threshold/resting potential states.

- Essential for preserving representations in dlPFC for delay cells.



# Establishing the basics

A simple model inclduing a neuron group: a spine on a dendrite attached to a soma

**Goal:** try to replicate spontaneous voltage potential in spine only.

In [21]:
# Morphology Params

d_spine_head = 0.4 # um
l_spine_neck = 0.4 # um
d_spine_neck = 0.1 # um

d_soma = 30 # um
d_dendrite = 1 # um
l_dendrite = 100 # um

morph = Soma(diameter = d_soma*um)
morph.dendrite = Cylinder(diameter = d_dendrite*um, length = l_dendrite*um, n = 1)
morph.dendrite.spine = Cylinder(diameter = d_spine_neck*um, length = l_spine_neck*um, n = 1)
morph.dendrite.spine.head = Soma(diameter = d_spine_head*um)

morph.topology()

( )  [root] 
   `-|  .dendrite
      `-|  .dendrite.spine
         `( )  .dendrite.spine.head

In [22]:
# Task: figure out a good way to visualize quickly

new_morpho = morph.generate_coordinates()

The key state variable, which must be specified at construction, is **Im**. It is the total transmembrane current, expressed in units of current per area. This is a mandatory line in the definition of the model.

At every timestep, Brian integrates the state variables, calculates the transmembrane current at every point on the neuronal morphology, and updates v using the transmembrane current and the diffusion current, which is calculated based on the morphology and the intracellular resistivity.

In [23]:
gL = 1e-4*siemens/cm**2
EL = -70*mV
Cm=1*uF/cm**2 # specific membrane capacitance
Ri=100*ohm*cm # intracellular resistivity

# Typical HH equations
eqs = '''
Im = gL * (EL - v) : amp/meter**2
Is = gS * (Es - v) : amp (point current)
dgs/dt = -gs/taus : siemens
'''

In [24]:
neuron = SpatialNeuron(morphology=morph, model=eqs, Cm=1*uF/cm**2, Ri=100*ohm*cm,)

In [25]:
spine_head = neuron.dendrite.spine.head

In [ ]:
neuron.v = EL
taum = Cm / gL # membrane time constant
la = neuron.space_constant[0]

N = 400
# Monitors
mon = StateMonitor(neuron, 'v', record = range(0, N//2, 20)